In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train = pd.read_csv('../input/ventilator-pressure-prediction/train.csv')
sample_submission = pd.read_csv('../input/ventilator-pressure-prediction/sample_submission.csv')
test = pd.read_csv('../input/ventilator-pressure-prediction/test.csv')

In [ ]:
train

In [ ]:
test

In [ ]:
sample_submission

In [ ]:
train.info()

In [ ]:
train.describe()

In [ ]:
train.isnull().any()

In [ ]:
train.corr()

In [ ]:
def add_features(df):
    df["u_in_sum"] = df.groupby("breath_id")["u_in"].transform("sum")
    df["u_in_cumsum"] = df.groupby("breath_id")["u_in"].cumsum()
    df["u_in_std"] = df.groupby("breath_id")["u_in"].transform("std")
    df["u_in_min"] = df.groupby("breath_id")["u_in"].transform("min")
    df["u_in_max"] = df.groupby("breath_id")["u_in"].transform("max")
    df["u_in_cumsum_reverse"] = df["u_in_sum"] - df["u_in_cumsum"]
    
    df["u_in_first"] = df.groupby("breath_id")["u_in"].transform("first")
    df["u_in_last"]  = df.groupby("breath_id")["u_in"].transform("last")
    
    df['time_diff']  = (df['time_step']).groupby(df['breath_id']).diff(1)
    df['time_diff2'] = (df['time_step']).groupby(df['breath_id']).diff(2)
    df['time_diff3'] = (df['time_step']).groupby(df['breath_id']).diff(3)
    df['time_diff4'] = (df['time_step']).groupby(df['breath_id']).diff(4)
    df['time_diff5'] = (df['time_step']).groupby(df['breath_id']).diff(5)
    df['time_diff6'] = (df['time_step']).groupby(df['breath_id']).diff(6)
    df['time_diff7'] = (df['time_step']).groupby(df['breath_id']).diff(7)
    df['time_diff8'] = (df['time_step']).groupby(df['breath_id']).diff(8)
    
    df["u_in_lag1"] = df.groupby("breath_id")["u_in"].shift(1)
    df["u_in_lead1"] = df.groupby("breath_id")["u_in"].shift(-1)
    df["u_in_lag1_diff"] = df["u_in"] - df["u_in_lag1"]
    df["u_in_lead1_diff"] = df["u_in"] - df["u_in_lead1"]
        
    df["time_passed"] = df.groupby("breath_id")["time_step"].diff()
    
    df['u_in_lag1']  = df.groupby('breath_id')['u_in'].shift(1)
    df['u_in_lag2']  = df.groupby('breath_id')['u_in'].shift(2)
    df['u_in_lag3']  = df.groupby('breath_id')['u_in'].shift(3)
    df['u_in_lag4']  = df.groupby('breath_id')['u_in'].shift(4)
    df['u_in_lag5']  = df.groupby('breath_id')['u_in'].shift(5)
    df['u_in_lag6']  = df.groupby('breath_id')['u_in'].shift(6)
    df['u_in_lag7']  = df.groupby('breath_id')['u_in'].shift(7)
    df['u_in_lag8']  = df.groupby('breath_id')['u_in'].shift(8)
    df['u_in_lag9']  = df.groupby('breath_id')['u_in'].shift(9)
    df['u_in_lag10'] = df.groupby('breath_id')['u_in'].shift(10)
    df['u_in_lag11'] = df.groupby('breath_id')['u_in'].shift(11)
    df['u_in_lag12'] = df.groupby('breath_id')['u_in'].shift(12)
    df['u_in_lag13'] = df.groupby('breath_id')['u_in'].shift(13)
    df['u_in_lag14'] = df.groupby('breath_id')['u_in'].shift(14)
    df['u_in_lag15'] = df.groupby('breath_id')['u_in'].shift(15)
    df['u_in_lag16'] = df.groupby('breath_id')['u_in'].shift(16)
    df['u_in_lag17'] = df.groupby('breath_id')['u_in'].shift(17)
    df['u_in_lag18'] = df.groupby('breath_id')['u_in'].shift(18)
    df['u_in_lag19'] = df.groupby('breath_id')['u_in'].shift(19)
    df['u_in_lag20'] = df.groupby('breath_id')['u_in'].shift(20)
    df['u_in_lag21'] = df.groupby('breath_id')['u_in'].shift(21)
    
    return df

In [ ]:
def prepare_data(df):
    bins = [
        -0.9, 1.0,
        1.1, 1.2, 1.3, 1.4, 1.5, 1.6, 1.7, 1.8, 1.9, 2.0,
        2.1, 2.2, 2.3, 2.4, 2.5, 2.6, 2.7, 2.8, 3
           ]
    labels = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
    df['time_step'] = pd.cut(x=df['time_step'], bins=bins, labels=labels)
    df['time_step'] = df['time_step'].astype('int64')
    train_R = pd.get_dummies(df['R'], prefix='R')
    train_C = pd.get_dummies(df['C'], prefix='C')
    train_u_out = pd.get_dummies(df['u_out'], prefix='u_out')
    df = pd.concat([df, train_R, train_C, train_u_out], axis=1)
    df = df.drop(['R', 'C', 'u_out'], axis=1)
    df = add_features(df)
    df = df.fillna(df.min())
    if 'pressure' in df:
        df['pressure'] = np.where(df['pressure'] < 0, 0, df['pressure'])
    return df

In [ ]:
train = prepare_data(train)
train

In [ ]:
X = train.drop('pressure', axis=1)
y = train['pressure']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

In [ ]:
from xgboost import XGBRegressor

xgb_params = {
    'n_estimators': 10000,  # initially - 5000
    'learning_rate': 0.1,  #initially - 0.1
    'subsample': 0.95,
    'colsample_bytree': 0.11,
    'max_depth': 5,  #initially - 2
    'booster': 'gbtree', 
    'reg_lambda': 66.1,
    'reg_alpha': 15.9,
    'random_state':42,
    'verbosity':1,
    'tree_method':'gpu_hist',
    'gpu_id':0,
    'predictor':'gpu_predictor'
}

# xgb_params = {'learning_rate':0.1, 'n_estimators':10000, 'max_depth':9,
#                 'n_jobs':16, 'tree_method':'gpu_hist', 'gpu_id':0, 'predictor':'gpu_predictor'}

model = XGBRegressor(**xgb_params)

model.fit(X_train,y_train)


In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_pred)

In [ ]:
model.save_model("xgb.model_1")

In [ ]:
train_R = []
train_C = []
train_u_out = []
y_test = []
y_pred = []
X_test = []
X = []
y = []
train = []

In [ ]:
test = prepare_data(test)
test

In [ ]:
# pred_y = model.predict(test)

In [ ]:
pred = model.predict(test)

In [ ]:
sample = pd.read_csv('../input/ventilator-pressure-prediction/sample_submission.csv')
sample['id'] = test['id']
sample['pressure'] = pd.DataFrame({'pressure': pred[:]})
# sample['pressure'] = np.square(np.square(sample['pressure']))
sample.head()

In [ ]:
q1 = sample['pressure'].quantile(0.001)
q2 = sample['pressure'].quantile(0.999)
sample['pressure'] = sample['pressure'].apply(lambda x: x if x>q1 else x*0.77)
sample['pressure'] = sample['pressure'].apply(lambda x: x if x<q2 else x*1.1)
sample.to_csv('submission.csv', index=False)

In [ ]:
pred

In [ ]:
test_R = []
test_C = []
test_u_out = []
pred = []
sample = []
test = []

In [ ]:
from xgboost import XGBRegressor

xgb_params = {
    'n_estimators': 7000,  # initially - 5000
    'learning_rate': 0.1,  #initially - 0.1
    'subsample': 0.95,
    'colsample_bytree': 0.11,
    'max_depth': 5,  #initially - 2
    'booster': 'gbtree', 
    'reg_lambda': 66.1,
    'reg_alpha': 15.9,
    'random_state':42,
    'verbosity':1,
    'tree_method':'gpu_hist',
    'gpu_id':0,
    'predictor':'gpu_predictor'
}

# xgb_params = {'learning_rate':0.1, 'n_estimators':10000, 'max_depth':9,
#                 'n_jobs':16, 'tree_method':'gpu_hist', 'gpu_id':0, 'predictor':'gpu_predictor'}

model_test = XGBRegressor(**xgb_params)

In [ ]:
model_test.load_model("./xgb.model_1")

In [ ]:
train = pd.read_csv('../input/ventilator-pressure-prediction/train.csv')

In [ ]:
X = train.drop('pressure', axis=1)
y = train['pressure']

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.10, random_state=42)

In [ ]:
X_test = prepare_data(X_test)
X_test

In [ ]:
y_pred = model_test.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, y_pred)

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(y_test, np.square(np.square(y_pred)))

In [ ]:
from sklearn.metrics import mean_squared_error
mean_squared_error(np.square(np.square(y_test)), np.square(np.square(y_pred)))